# Bilan Hydrique

Sources :

- Coefficients culturaux :
  - [ARDEPI](https://www.ardepi.fr/nos-services/vous-etes-irrigant/estimer-ses-besoins-en-eau/maraichage/)
  - [Chambre d’agriculture Nouvelle-Aquitaine](https://gironde.chambre-agriculture.fr/fileadmin/user_upload/Nouvelle-Aquitaine/100_Inst-Gironde/Documents/pdf_grandes-cultures_accompagnement-technique_mieux-irriguer/Messages_irrigation_2019/message_1/Tableau_Coefficients_Culturaux_Kc_.02.pdf)


In [32]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [33]:
# Constantes
# Coefficients culturaux (KC) par culture et par stade
KC = {
    'Pomme de terre': {'Vegetation': 0.9, 'Maximale': 1.05}
}

# Réserve Utile (RU) par cm de terre fine (mm/cm de terre fine) en fonction de la texture du sol
RU_PAR_CM_DE_TF = {
    'Terres argileuses': 1.85,
    'Argiles sableuses': 1.7, 'Argiles sablo-limoneuses': 1.8, 'Argiles limono-sableuses': 1.8, 'Argiles limoneuses': 1.9,
    'Terres argilo-sableuses': 1.7, 'Terres argilo-limono-sableuses': 1.8, 'Terres argilo-limoneuses': 2.,
    'Terres sablo-argileuses': 1.4, 'Terres sablo-limono-argileuses': 1.5, 'Terres limono-sablo-argileuses': 1.65, 'Terres limono-argileuses': 2.00,
    'Terres sableuses': 0.7, 'Terres sablo-limoneuses': 1., 'Terres limono-sableuses': 1.55, 'Terres limoneuses': 1.8
}

# Profondeurs d'enracinement typiques
PROFONDEUR_ENRACINEMENT_TYPIQUE = {
    'Radis': 15.,
    'Salade': 15.,
    'Choux': 20.,
    'Epinard': 20.,
    'Oignon': 20.,
    'Aubergine': 30.,
    'Carotte': 30.,
    'Courge': 30.,
    'Courgette': 30.,
    'Poivron': 30.,
    'Pomme de terre': 30.,
    'Tomate': 30.
}   

In [34]:
# Données météorologiques
# Évapotranspiration Potentielle (ETP) sur la période (mm)
ETP = 28.

# Réserve Facilement Utilisable (RFU) initiale (mm)
RFU_INITIALE = 40.

# Pluviométrie (mm)
PLUVIOMETRIE = 5.

In [35]:
# RFU finale cible (mm)
RFU_FINALE_CIBLE = 40.

# Coefficient de conversion de la RU en RFU (entre 1/2 et 2/3)
RU_VERS_RFU = 2. / 3

# Fraction de la réserve utile du sol remplie d'eau (entre 0 pour une période sèche et 1 pour une période pluvieuse)
FRACTION_REMPLIE = 1.

# Fraction du sol occupé par des cailloux et graviers (entre 0 pour absence de cailloux et 1 pour totalité de cailloux)
FRACTION_CAILLOUX = 0.3

In [36]:
# Choix de la culture
culture = 'Pomme de terre'

# Choix du stade
stade = 'Vegetation'

# Choix de la texture
texture = 'Terres limoneuses'

In [37]:
# RU par cm de terre fine pour cette texture
ru_par_cm_de_tf_texture = RU_PAR_CM_DE_TF[texture]

# Calcul de la profondeur de terre fine
profondeur = PROFONDEUR_ENRACINEMENT_TYPIQUE[culture]
profondeur_tf = profondeur * (1. - FRACTION_CAILLOUX)

# Calcul de la RU (mm)
ru = ru_par_cm_de_tf_texture * profondeur_tf

# Calcul de la RFU maximale (mm)
rfu_max = ru * RU_VERS_RFU

# Calcul de la RFU disponible (mm)
rfu = rfu_max * FRACTION_REMPLIE

print(f"RFU pour une profondeur d'enracinement de {profondeur:.0f} cm : {rfu:.0f} mm")

RFU pour une profondeur d'enracinement de 30 cm : 25 mm


In [38]:
# KC de la culture pour ce stade
kc_culture = KC[culture][stade]

# Calcul de l'évalotranspiration maximale (mm)
etm_culture = kc_culture * ETP

print(f'Évapotranspiration maximale pour la {culture} au stade {stade}: {etm_culture:.0f} (mm)')

Évapotranspiration maximale pour la Pomme de terre au stade Vegetation: 25 (mm)


In [39]:
# Calcul de la RFU finale
besoin_irrigation = RFU_FINALE_CIBLE + etm_culture - (rfu + PLUVIOMETRIE)

print(f'Besoin en irrigation pour la {culture} au stade {stade}: {besoin_irrigation:.0f} (mm)')

Besoin en irrigation pour la Pomme de terre au stade Vegetation: 35 (mm)


In [40]:
# Définition de la station de référence
REF_STATION_NAME = 'La Petite Claye'
REF_STATION_LATLON = [48.541356, -1.615400]

In [93]:
from io import StringIO
import json
import requests

# Definitions pour accéder à l'API Météo-France via un token OAuth2
# unique application id : you can find this in the curl's command to generate jwt token 
APPLICATION_ID = 'ZlFGb1VCNzdlQ3c5QmhSMU1IbE8xQTluOE0wYTpUS3l1YkcweGJmSTJrQlJVaGNiSkNHTXczdHNh'

# url to obtain acces token
TOKEN_URL = "https://portail-api.meteofrance.fr/token"

class Client(object):
    def __init__(self):
        self.session = requests.Session()
        
    def request(self, method, url, **kwargs):
        # First request will always need to obtain a token first
        if 'Authorization' not in self.session.headers:
            self.obtain_token()
            
        # Optimistically attempt to dispatch reqest
        response = self.session.request(method, url, **kwargs)

        if self.token_has_expired(response):
            # We got an 'Access token expired' response => refresh token
            self.obtain_token()

            # Re-dispatch the request that previously failed
            response = self.session.request(method, url, **kwargs)

        return response

    def token_has_expired(self, response):
        status = response.status_code
        content_type = response.headers['Content-Type']
        repJson = response.text

        if status == 401 and 'application/json' in content_type:
            repJson = response.text
            
            if 'Invalid JWT token' in repJson['description']:
                return True

        return False

    def obtain_token(self):
        # Obtain new token
        data = {'grant_type': 'client_credentials'}
        headers = {'Authorization': 'Basic ' + APPLICATION_ID}
        access_token_response = requests.post(
            TOKEN_URL, data=data, verify=False, allow_redirects=False, headers=headers)
        token = access_token_response.json()['access_token']

        # Update session with fresh token
        self.session.headers.update({'Authorization': 'Bearer %s' % token})

def response_text_to_frame(response, **read_csv_kwargs):
    return pd.read_csv(StringIO(response.text), sep=';', **read_csv_kwargs)

In [94]:
# Initialisation d'un client pour accéder à l'API Météo-France
client = Client()

# Issue a series of API requests an example. For use this test, you must first subscribe to the arome api with your application
client.session.headers.update({'Accept': '*/*'})

In [95]:
# URL de la liste des stations
URL_LISTE_STATIONS = 'https://public-api.meteofrance.fr/public/DPObs/v1/liste-stations'

# Étiquettes de la latitude et de la longitude
LATLON_LABELS = ['Latitude', 'Longitude']

In [96]:
# Demande de la liste des stations
response_liste_stations = client.request('GET', URL_LISTE_STATIONS, verify=False)

/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'portail-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [97]:
# Conversion du texte répondu en DataFrame
df_liste_stations = response_text_to_frame(response_liste_stations, index_col='Nom_usuel')

In [98]:
# Convert degrees to radians
latlon_rad_labels = []
for station_name, station_series in df_liste_stations[LATLON_LABELS].items():
    coord_label_rad = f'{station_name}_rad'
    df_liste_stations[coord_label_rad] = np.deg2rad(station_series)
    latlon_rad_labels.append(coord_label_rad)
ref_station_latlon_rad = np.deg2rad(REF_STATION_LATLON)

In [99]:
from sklearn.neighbors import BallTree

# Calcul de l'arbre des plus proches voisins pour la liste des stations
X = df_liste_stations[latlon_rad_labels].values
tree = BallTree(X, metric='haversine')

In [100]:
# Rayon de la terre (km)
RAYON_TERRE_KM = 6371.

# # Identification d'un certain nombre de plus proches voisins
# NN_NUMBER = 20
# dist_rad_arr, ind_arr = tree.query([ref_station_latlon_rad], k=NN_NUMBER)

# Alternative : identification des plus proches voisins dans un certain rayon
NN_RAYON_KM = 40.
nn_rayon_rad = NN_RAYON_KM / RAYON_TERRE_KM
ind_arr, dist_rad_arr = tree.query_radius([ref_station_latlon_rad], nn_rayon_rad,
                                          count_only=False, return_distance=True)

dist_rad, ind = dist_rad_arr[0], ind_arr[0]

# Conversion en km de la distance en rad
dist_km = np.round(dist_rad * RAYON_TERRE_KM).astype(int)

In [101]:
# Sélection des plus proches voisins
df_liste_stations_nn = df_liste_stations.iloc[ind]
print(f'{REF_STATION_NAME} ({REF_STATION_LATLON[0]:.6f}, {REF_STATION_LATLON[1]:.6f}) est à :')
for k, (nn_station_name, nn_station_series) in enumerate(df_liste_stations_nn.transpose().items()):
    print(f'- {dist_km[k]:02d} km du {k}NN {nn_station_name} '
          f'({nn_station_series[LATLON_LABELS[0]]:.6f}, {nn_station_series[LATLON_LABELS[1]]:.6f})')

La Petite Claye (48.541356, -1.615400) est à :
- 17 km du 0NN PLERGUER (48.524833, -1.843667)
- 27 km du 1NN PLESDER (48.406833, -1.924833)
- 34 km du 2NN DINARD (48.584833, -2.076333)
- 24 km du 3NN FEINS  SA (48.326833, -1.596833)
- 06 km du 4NN BROUALAN (48.485667, -1.640833)
- 33 km du 5NN GRANVILLE (48.834500, -1.613667)
- 36 km du 6NN LONGUEVILLE (48.862000, -1.573000)
- 09 km du 7NN PONTORSON (48.585667, -1.505167)
- 31 km du 8NN ST OVIN (48.682500, -1.248667)
- 36 km du 9NN LOUVIGNE-DU-DESERT (48.479333, -1.129833)
- 37 km du 10NN FOUGERES (48.337167, -1.212667)
- 29 km du 11NN MEZIERES-SUR-C. (48.308833, -1.439000)
- 37 km du 12NN EQUILLY (48.834500, -1.388667)
- 38 km du 13NN ST-HILAIRE-DU-H (48.563667, -1.096667)


In [104]:
# URL de base des données horaires
URL_DONNEE_HORAIRE = 'https://public-api.meteofrance.fr/public/DPObs/v1/station/horaire'

# Format des donnée renvoyée
FMT = 'csv'

def demande_donnee_station_date(id_station, date, verify=False):
    '''Demande des données horaires pour une station et une date données.'''
    # Paramètres définissant la station, la date et le format des données
    params_station_date = {'id_station': id_station, 'date': date, 'format': FMT}

    # Demande de la liste des stations
    response_station = client.request(
        'GET', URL_DONNEE_HORAIRE, verify=verify, params=params_station_date)

    return response_station

def response_station_to_series(response_station):
    '''Conversion du texte répondu en DataFrame.'''
    df_station = response_text_to_frame(response_station)
    s_station = df_station.iloc[0]

    return s_station

def compiler_donnees_liste_stations_date(df_liste_stations, date):
    df_donnees_liste_stations = pd.DataFrame(dtype=float)
    for station_name in df_liste_stations.index:
        # Identifiant de la station
        id_station = int(df_liste_stations.loc[station_name]['Id_station'])
        response_station = demande_donnee_station_date(id_station, date)
        s_station = response_station_to_series(response_station)
        df_donnees_liste_stations = pd.concat([
            df_donnees_liste_stations, s_station.to_frame(station_name).transpose()])
    return df_donnees_liste_stations

In [105]:
# Choix de la date
DATE = '2025-01-02T14:00:00Z'

df_donnees_liste_stations = compiler_donnees_liste_stations_date(df_liste_stations_nn, date)

/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/atantet/.conda/envs/agri_notebooks/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'public-api.meteofrance.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/late

,lat,lon,geo_id_insee,reference_time,insert_time,validity_time,t,td,tx,tn,...,t_50,t_100,vv,etat_sol,sss,n,insolh,ray_glo01,pres,pmer
PLERGUER,48.524833,-1.843667,35224001,2025-01-02T14:10:08Z,2025-01-02T14:06:15Z,2025-01-02T14:00:00Z,280.95,NaN,281.25,280.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PLESDER,48.406833,-1.924833,35225001,2025-01-02T14:10:08Z,2025-01-02T14:06:15Z,2025-01-02T14:00:00Z,281.05,NaN,281.15,280.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DINARD,48.584833,-2.076333,35228001,2025-01-02T14:10:08Z,2025-01-02T14:02:15Z,2025-01-02T14:00:00Z,280.85,277.75,281.05,280.85,...,282.45,NaN,47460,0,0,8,0,321000,101150,101950
FEINS SA,48.326833,-1.596833,35110003,2025-01-02T14:10:08Z,2025-01-02T14:02:15Z,2025-01-02T14:00:00Z,280.55,277.55,280.75,280.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BROUALAN,48.485667,-1.640833,35044001,2025-01-02T14:10:08Z,2025-01-02T14:05:15Z,2025-01-02T14:00:00Z,280.25,NaN,280.55,280.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GRANVILLE,48.8345,-1.613667,50218001,2025-01-02T14:10:08Z,2025-01-02T14:08:45Z,2025-01-02T14:00:00Z,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LONGUEVILLE,48.862,-1.573,50277001,2025-01-02T14:10:08Z,2025-01-02T14:01:16Z,2025-01-02T14:00:00Z,280.35,NaN,280.55,280.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PONTORSON,48.585667,-1.505167,50410003,2025-01-02T14:10:08Z,2025-01-02T14:01:15Z,2025-01-02T14:00:00Z,280.75,277.55,281.05,280.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ST OVIN,48.6825,-1.248667,50531001,2025-01-02T14:10:08Z,2025-01-02T14:01:15Z,2025-01-02T14:00:00Z,279.35,NaN,279.45,279.35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LOUVIGNE-DU-DESERT,48.479333,-1.129833,35162003,2025-01-02T14:10:08Z,2025-01-02T14:01:15Z,2025-01-02T14:00:00Z,279.85,276.95,280.05,279.85,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [107]:
# Chaleur latente de vaporisation de l’eau (J kg-1)
LAMBDA = 2.45e6

# Constante psychométrique (kPa K-1)
GAMMA = 0.000665

# Constante de Stefan (W m-2 K-4)
SIGMA = 5.67e8 

# Albedo
ALPHA = 0.2

# Émissivité
EPSILON = 0.95

# ETP journalière maximale (mm j-1)
ETP_JOUR_MAX = 9.

# Rayonnement atmosphérique incident (J m-2)
RAYONNEMENT_ATMOSPHERIQUE = 0.

def calcul_etp(s_station):
    # Variable météorologiques
    # Vitesse du vent à 10 m (m s-1)
    vitesse_vent_10m = s_station['ff']
    print(f'Vitesse du vent à 10 m (m s-1) : {vitesse_vent_10m:.1f}')

    # Temperature de l'air à 2 m (K)
    temperature_air_2m = s_station['t']
    print(f"Temperature de l'air à 2 m (K) : {temperature_air_2m:.1f}")

    # Humidité relative de l'air à 2 m
    humidite_relative_air_2m = s_station['u'] / 100
    print(f"Humidité relative de l'air à 2 m : {humidite_relative_air_2m:.2f}")

    # Rayonnement global (J m-2)
    rayonnement_global = s_station['ray_glo01']
    print(f'Rayonnement global (J m-2) : {rayonnement_global:.1f}')

    # Calcul de la pente de la courbe de pression de vapeur à la température moyenne de l'air (kPa K-1)
    exposant = 17.27 * (temperature_air_2m - 273.15) / (temperature_air_2m - 35.85)
    delta = 2504 * np.exp(exposant) / (temperature_air_2m - 35.85)**2

    # Calcul de rho * Cp / ra
    rho_cp_sur_ra = LAMBDA * GAMMA * 0.26 * (1. + 0.4 * vitesse_vent_10m)

    # Calcul de la pression de vapeur saturante (kPa)
    es = 0.6108 * np.exp(exposant)

    # Calcul de la pression de vapeur effective (kPa)
    ee = humidite_relative_air_2m * es

    # Calcul du rayonnement net
    rayonnement_net = ((1 - ALPHA) * rayonnement_global +
                       EPSILON * RAYONNEMENT_ATMOSPHERIQUE -
                       EPSILON * SIGMA * temperature_air_2m**4)

    # Calcul de l'ETP (mm h-1)
    denominateur = LAMBDA * (delta + GAMMA)
    etp1 = max(0, delta * rayonnement_net / denominateur)
    etp2 = max(0, rho_cp_sur_ra * (es - ee) / denominateur)
    etp = etp1 + etp2

    return etp

In [108]:
s_station = df_donnees_liste_stations.loc['DINARD']

etp = calcul_etp(s_station)

# Calcul de l'ETP journalière (mm j-1)
etp_jour = min(ETP_JOUR_MAX, etp * 24)

Vitesse du vent à 10 m (m s-1) : 2.9
Temperature de l'air à 2 m (K) : 280.9
Humidité relative de l'air à 2 m : 0.81
Rayonnement global (J m-2) : 321000.0


In [31]:
etp

np.float64(0.001029424383709898)